## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Castro,BR,-24.7911,-50.0119,64.09,94,24,2.93,few clouds
1,1,New Norfolk,AU,-42.7826,147.0587,46.99,88,100,2.77,overcast clouds
2,2,Makat,KZ,47.6469,53.3463,54.90,41,0,12.03,clear sky
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,82.40,61,0,9.22,clear sky
4,4,Birin,NP,26.8865,88.0718,59.59,17,0,6.02,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Arraial Do Cabo,BR,-22.9661,-42.0278,82.40,61,0,9.22,clear sky
12,12,Atuona,PF,-9.8000,-139.0333,80.91,74,40,16.31,scattered clouds
13,13,Kulhudhuffushi,MV,6.6221,73.0700,82.63,71,31,1.36,scattered clouds
16,16,Santa Marta,CO,11.2408,-74.1990,84.99,72,38,11.99,scattered clouds
19,19,Rikitea,PF,-23.1203,-134.9692,78.46,69,4,9.69,clear sky
22,22,Madang,PG,-5.2246,145.7966,77.67,89,98,0.89,overcast clouds
24,24,Upata,VE,8.0086,-62.3989,87.01,71,94,5.01,overcast clouds
25,25,Mahebourg,MU,-20.4081,57.7000,75.20,94,40,1.14,moderate rain
31,31,Najran,SA,17.4924,44.1277,77.00,23,20,4.59,few clouds
33,33,Hilo,US,19.7297,-155.0900,78.80,57,1,11.50,light rain


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Arraial Do Cabo,BR,82.40,clear sky,-22.9661,-42.0278,
12,Atuona,PF,80.91,scattered clouds,-9.8000,-139.0333,
13,Kulhudhuffushi,MV,82.63,scattered clouds,6.6221,73.0700,
16,Santa Marta,CO,84.99,scattered clouds,11.2408,-74.1990,
19,Rikitea,PF,78.46,clear sky,-23.1203,-134.9692,
22,Madang,PG,77.67,overcast clouds,-5.2246,145.7966,
24,Upata,VE,87.01,overcast clouds,8.0086,-62.3989,
25,Mahebourg,MU,75.20,moderate rain,-20.4081,57.7000,
31,Najran,SA,77.00,few clouds,17.4924,44.1277,
33,Hilo,US,78.80,light rain,19.7297,-155.0900,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [28]:
# 7. Drop the rows where there is no Hotel Name.
# Replace blank cells under hotel names with NaN
hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
# Drop rows with NaN in Hotel Name. 
clean_hotel_df = hotel_df.dropna()

In [29]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [34]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [35]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))